In [27]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import GRU, Activation, Dropout, Dense, Input, Bidirectional, Layer
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adadelta

from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from tensorflow import keras
import keras.backend as kb

import sys
sys.path.append("../")

from pre_processing import creazione_modello_GloVe

In [28]:
dataset = pickle.load(open("dataset/dataset_SLS.pickle", "rb"))

In [29]:
Y = np.array(dataset["sentiment"])

In [30]:
x_train,x_test,y_train,y_test = train_test_split(dataset["sentence"],Y, test_size=0.1, shuffle=True)

In [31]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(2700,) (2700,)
(300,) (300,)


In [32]:
print("numero di positivi nel train: ", list(y_train).count(1))
print("numero di negativi nel train: ", list(y_train).count(0))

print("numero di positivi nel test: ", list(y_test).count(1))
print("numero di negativi nel test: ", list(y_test).count(0))

numero di positivi nel train:  1360
numero di negativi nel train:  1340
numero di positivi nel test:  140
numero di negativi nel test:  160


In [33]:
embedding = creazione_modello_GloVe("dataset/glove.6B.50d.txt")

In [34]:
tokenizer = Tokenizer(num_words=140000)
tokenizer.fit_on_texts(x_train)

word_index = tokenizer.word_index

In [35]:
vocab_len = len(word_index)+1
embedding_vector_len = embedding["banana"].shape[0]
embedding_matrix = np.zeros((vocab_len, embedding_vector_len))

for word, index in word_index.items():
    vector = embedding.get(word)
    if vector is not None:
        embedding_matrix[index, :] = vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embedding_vector_len, input_length=300, weights=[embedding_matrix])

In [36]:
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=kb.squeeze(kb.tanh(kb.dot(x,self.W)+self.b),axis=-1)
        at=kb.softmax(et)
        at=kb.expand_dims(at,axis=-1)
        output=x*at
        return kb.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [60]:
BiGRUAtt_model = Sequential()
BiGRUAtt_model.add(embedding_layer)
BiGRUAtt_model.add(Bidirectional(GRU(units=16, return_sequences=True)))

BiGRUAtt_model.add(Dense(16))
BiGRUAtt_model.add(Dense(16))

BiGRUAtt_model.add(attention())

BiGRUAtt_model.add(Dense(2, activation="softmax"))

BiGRUAtt_model.compile(loss="categorical_crossentropy", optimizer=Adadelta(), metrics=["accuracy"])
BiGRUAtt_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 300, 50)           249800    
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 300, 32)           6528      
_________________________________________________________________
dense_40 (Dense)             (None, 300, 16)           528       
_________________________________________________________________
dense_41 (Dense)             (None, 300, 16)           272       
_________________________________________________________________
attention_11 (attention)     (None, 16)                316       
_________________________________________________________________
dense_42 (Dense)             (None, 2)                 34        
Total params: 257,478
Trainable params: 257,478
Non-trainable params: 0
_______________________________________________

In [38]:
x_train_index = tokenizer.texts_to_sequences(x_train)
x_train_index = pad_sequences(x_train_index, maxlen=300)

In [39]:
y_train_categorical = keras.utils.to_categorical(y_train, 2)

In [55]:
BiGRUAtt_model.fit(x_train_index[:2100], y_train_categorical[:2100], epochs=20, batch_size=64, verbose=1, validation_data=(x_train_index[2100:], y_train_categorical[2100:]))

Epoch 1/20
33/33 [==============================] - 10s 223ms/step - loss: 0.6903 - accuracy: 0.5233 - val_loss: 0.6807 - val_accuracy: 0.6117
Epoch 2/20
33/33 [==============================] - 7s 207ms/step - loss: 0.6507 - accuracy: 0.6367 - val_loss: 0.5888 - val_accuracy: 0.7450
Epoch 3/20
33/33 [==============================] - 7s 208ms/step - loss: 0.4971 - accuracy: 0.7781 - val_loss: 0.4265 - val_accuracy: 0.8433
Epoch 4/20
33/33 [==============================] - 7s 217ms/step - loss: 0.3906 - accuracy: 0.8381 - val_loss: 0.3818 - val_accuracy: 0.8483
Epoch 5/20
33/33 [==============================] - 8s 248ms/step - loss: 0.3100 - accuracy: 0.8771 - val_loss: 0.3420 - val_accuracy: 0.8550
Epoch 6/20
33/33 [==============================] - 7s 221ms/step - loss: 0.2476 - accuracy: 0.9095 - val_loss: 0.3423 - val_accuracy: 0.8717
Epoch 7/20
33/33 [==============================] - 7s 198ms/step - loss: 0.1957 - accuracy: 0.9281 - val_loss: 0.3819 - val_accuracy: 0.8533
Epoch

In [56]:
x_test_index = tokenizer.texts_to_sequences(x_test)
x_test_index = pad_sequences(x_test_index, maxlen=300)

In [57]:
y_pred = BiGRUAtt_model.predict(x_test_index)

In [58]:
print(classification_report(y_test, np.argmax(y_pred, axis=1).astype("float32")))

              precision    recall  f1-score   support

           0       0.90      0.82      0.86       160
           1       0.82      0.89      0.85       140

    accuracy                           0.86       300
   macro avg       0.86      0.86      0.86       300
weighted avg       0.86      0.86      0.86       300



In [59]:
BiGRUAtt_model.save("models/BiGRUAtt")

INFO:tensorflow:Assets written to: models/BiGRUAtt/assets
INFO:tensorflow:Assets written to: models/BiGRUAtt/assets
